# Assignment 7

## Submit as an HTML file

### Print your name below

In [2]:
print('Everly Wang')

Everly Wang


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [31]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf



#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [9]:
result=pd.read_csv('data/results.csv')

display(result.head())

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [12]:
print(result.dtypes)
result["race_time_ms"]=pd.to_numeric(result["milliseconds"], errors='coerce')




resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
dtype: object


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [15]:
result["finish_category"]=pd.cut(result["positionOrder"], bins=[0, 3,10,20,float('inf')], 
    labels=['Podium', 'Points', 'Midfield', 'Backmarker'],right=True)


### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [22]:
result["race_duration_minutes"]=result["race_time_ms"]/60/1000

constructor=(result.groupby(  "constructorId"  )
                       .agg(average_duration = ('race_duration_minutes','mean'))
                       .sort_values(by='average_duration', ascending=True)
                       .head(5).reset_index()

)

display(constructor)



,constructorId,average_duration
0,35,76.710777
1,29,77.604125
2,41,87.046767
3,16,89.428828
4,53,89.658852


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [24]:
top_driver=(result.groupby(  "driverId"  )
                          .agg(average_position = ('positionOrder','mean'),
                               total_points = ('points','sum'),
                               number_of_races=('raceId','count'),
                               best_finish=('positionOrder','min')
                               ).sort_values(by='total_points', ascending=False)
                               .head(10).reset_index()
)

display(top_driver)

,driverId,average_position,total_points,number_of_races,best_finish
0,1,4.787097,4396.5,310,1
1,20,7.093333,3098.0,300,1
2,4,8.494413,2061.0,358,1
3,830,6.533742,1983.5,163,1
4,8,8.491477,1873.0,352,1
5,822,7.601990,1778.0,201,1
6,3,8.252427,1594.5,206,1
7,30,6.879870,1566.0,308,1
8,817,9.883621,1307.0,232,1
9,18,9.695793,1235.0,309,1


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [33]:
result_clean=result.dropna(subset=['points','grid', "laps"])

model = smf.ols(formula='points ~ grid', data=result_clean) 
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.136
Method:                 Least Squares   F-statistic:                     4074.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        20:14:45   Log-Likelihood:                -71669.
No. Observations:               25840   AIC:                         1.433e+05
Df Residuals:                   25838   BIC:                         1.434e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.2521      0.044     95.905      0.0